In [1]:
import pandas as pd
import numpy as np
import os
import sys
data_dir = os.getcwd()+"/log_data/"
sys.argv = ["file","train.csv", "test.csv", "param_a.txt" ,"outputfile.csv", "weightfile.csv"]

In [2]:
def hotencode_train(data):
    train = pd.DataFrame()
    parents = pd.get_dummies(data[0], prefix="parents")
    has_nurs = pd.get_dummies(data[1], prefix="has_nurs")
    form = pd.get_dummies(data[2], prefix="form")
    children = pd.get_dummies(data[3], prefix="children")
    housing = pd.get_dummies(data[4], prefix="housing")
    finance = pd.get_dummies(data[5], prefix="finance")
    social = pd.get_dummies(data[6], prefix="social")
    health = pd.get_dummies(data[7], prefix="health")
    classDistribution = pd.get_dummies(data[8], prefix="class")
    train = pd.concat([parents, has_nurs, form, children, housing,
                    finance, social, health, classDistribution], axis=1)
    cols = ['parents_usual', 'parents_pretentious', 'parents_great_pret', 'has_nurs_proper', 'has_nurs_less_proper', 'has_nurs_improper',
            'has_nurs_critical', 'has_nurs_very_crit',
            'form_complete', 'form_completed', 'form_incomplete', 'form_foster',
            'children_1', 'children_2', 'children_3', 'children_more',
            'housing_convenient', 'housing_less_conv', 'housing_critical',
            'finance_convenient', 'finance_inconv',
            'social_nonprob', 'social_slightly_prob', 'social_problematic',
            'health_recommended', 'health_priority', 'health_not_recom',
            'class_not_recom', 'class_recommend', 'class_very_recom', 'class_priority', 'class_spec_prior']

    return train[pd.Index(cols)]

In [3]:
def hotencode_test(data):
    train = pd.DataFrame()
    parents = pd.get_dummies(data[0], prefix="parents")
    has_nurs = pd.get_dummies(data[1], prefix="has_nurs")
    form = pd.get_dummies(data[2], prefix="form")
    children = pd.get_dummies(data[3], prefix="children")
    housing = pd.get_dummies(data[4], prefix="housing")
    finance = pd.get_dummies(data[5], prefix="finance")
    social = pd.get_dummies(data[6], prefix="social")
    health = pd.get_dummies(data[7], prefix="health")
    train = pd.concat([parents, has_nurs, form, children, housing,
                    finance, social, health], axis=1)
    cols = ['parents_usual', 'parents_pretentious', 'parents_great_pret', 'has_nurs_proper', 'has_nurs_less_proper', 'has_nurs_improper',
            'has_nurs_critical', 'has_nurs_very_crit',
            'form_complete', 'form_completed', 'form_incomplete', 'form_foster',
            'children_1', 'children_2', 'children_3', 'children_more',
            'housing_convenient', 'housing_less_conv', 'housing_critical',
            'finance_convenient', 'finance_inconv',
            'social_nonprob', 'social_slightly_prob', 'social_problematic',
            'health_recommended', 'health_priority', 'health_not_recom']
    return train[pd.Index(cols)]

In [65]:
train_data = pd.read_csv(data_dir+sys.argv[1], header=None)
test_data = pd.read_csv(data_dir+sys.argv[2], header=None)
params = np.loadtxt(data_dir+sys.argv[3])
train_data = hotencode_train(train_data)
test_data = hotencode_test(test_data)

In [66]:
print(train_data.shape,test_data.shape)

(6000, 32) (2000, 27)


In [67]:
classes = train_data.iloc[:,-5:].values
features = train_data.iloc[:,:-5].values
features = np.c_[np.ones(len(train_data)),features]

In [71]:
X_train = features[:5400,:]
Y_train = classes[:5400,:]

In [69]:
testFeatures = test_data.iloc[:,:].values
testFeatures = np.c_[np.ones(len(test_data)),testFeatures]
X_test = features[:,:]
Y_test = classes[:,:]

In [27]:
# import seaborn as sn
# sn.heatmap(test_data.isnull(),yticklabels=False)
# train_data.isnull().sum()


In [28]:
print(X_train[:5400,:].shape,Y_train[:5400,:2].shape,X_test.shape)

(5400, 28) (5400, 2) (2000, 28)


In [12]:
print(X_train.shape,Y_train.shape,X_test.shape)
w_initial = np.ones(28*5).reshape(28, 5)

(6000, 28) (6000, 5) (2000, 28)


In [13]:
def gradient(X,Y,W):
    XW = np.exp(np.matmul(X, W))
    denom = np.sum(XW, axis=1)
    Y_predict = np.divide(XW, denom.reshape(X.shape[0], 1))
    return np.matmul(X.transpose(), Y - Y_predict)/X.shape[0]

In [73]:
def cost2(W,X,Y):
    XW = np.exp(np.matmul(X,W))
    denom = np.sum(XW, axis=1)
    Y_predict = np.divide(XW, denom.reshape(X.shape[0], 1))
    error = -np.trace(np.matmul(Y.T,np.log(Y_predict)))/X.shape[0]
    return err
 

In [61]:
def cost(W,X,Y):
    XW = np.exp(np.matmul(X,W))
    logTerm = np.log(np.sum(XW,axis=1))
    weightedTerm = np.sum(np.multiply(np.matmul(Y,W.T),X),axis=1)
    error = np.sum(logTerm-weightedTerm)/X.shape[0]
    return error

In [63]:
w_initial = np.ones(28*5).reshape(28, 5)
k =1
l = X_train.shape[0]
costAr = []
for j in range(1, 40000):
    grad = gradient(X_train,Y_train,w_initial)
    w_initial = w_initial-0.1*grad  
    c = cost(w_initial,X_train,Y_train)
    print((j,c), end="\r", flush=True)
    costAr.append(c)

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(0, len(costAr)),costAr)
plt.show()

In [ ]:
import numpy as np
alpha = 0.3
beta = 0.8

f = lambda x: (x[0]**2 + 3*x[1]*x[0] + 12)
dfx1 = lambda x: (2*x[0] + 3*x[1])
dfx2 = lambda x: (3*x[0])

t = 1
count = 1
x0 = np.array([2,3])
dx0 = np.array([.1, 0.05])


def backtrack(x0, dfx1, dfx2, t, alpha, beta, count):
    while (f(x0) - (f(x0 - t*np.array([dfx1(x0), dfx2(x0)])) + alpha * t * np.dot(np.array([dfx1(x0), dfx2(x0)]), np.array([dfx1(x0), dfx2(x0)])))) < 0:
        t *= beta
        print("""

########################
###   iteration {}   ###
########################
""".format(count))
        print("Inequality: ",  f(x0) - (f(x0 - t*np.array([dfx1(x0), dfx2(x0)])) + alpha * t * np.dot(np.array([dfx1(x0), dfx2(x0)]), np.array([dfx1(x0), dfx2(x0)]))))
        count += 1
    return t

t = backtrack(x0, dfx1, dfx2, t, alpha, beta,count)

print("\nfinal step size :",  t)

In [74]:
# part a
if(params[0] == 1):
    w_initial = np.ones(28*5).reshape(28, 5)
    w_initial = (w_initial.T - w_initial.T[0]).T
    costAr = []
    costAr.append(0)
    for j in range(1, int(params[2])):
        grad = gradient(X_train,Y_train,w_initial)
        w_initial = w_initial+params[1]*grad 
        w_initial = (w_initial.T - w_initial.T[0]).T
        c = cost(w_initial,X_train,Y_train)
        print((j,c), end="\r", flush=True)
        costAr.append(c)
        if(costAr[j-1]-costAr[j]<.000001 and j>10):
            print(costAr[j],costAr[j-1])
            break
if(params[0] == 2):
    w_initial = np.ones(28*5).reshape(28, 5)
    w_initial = (w_initial.T - w_initial.T[0]).T
    costAr = []
    costAr.append(0)
    for j in range(1, int(params[2])):
        grad = gradient(X_train,Y_train,w_initial)
        w_initial = w_initial+(params[1]/np.sqrt(j))*grad 
        w_initial = (w_initial.T - w_initial.T[0]).T
        c = cost(w_initial,X_train,Y_train)
        print((j,c), end="\r", flush=True)
        costAr.append(c)
        if(costAr[j-1]-costAr[j]<.000001 and j>20):
            print(costAr[j],costAr[j-1])
            break

KeyboardInterrupt: 

In [41]:
print(j)
w_initial = (w_initial.T - w_initial.T[0]).T
WmulX = np.exp(np.matmul(X_test, w_initial))
denom = np.sum(WmulX, axis=1)
Y_predict = np.divide(WmulX, denom.reshape(len(X_test), 1))
b = np.zeros_like(Y_predict)
b[np.arange(len(Y_predict)), Y_predict.argmax(1)] = 1
Y_predict = b
# print(Y_predict.shape)
np.trace(np.matmul(Y_predict, Y_test.T))/len(X_test)

3468


0.9133333333333333

In [42]:
Y_predict[:,0] =1*Y_predict[:,0]
Y_predict[:,1] =2*Y_predict[:,1]
Y_predict[:,2] =3*Y_predict[:,2]
Y_predict[:,3] =4*Y_predict[:,3]
Y_predict[:,4] =5*Y_predict[:,4]
Y_predict = np.sum(Y_predict,axis=1).tolist()
Y_test[:,0] =1*Y_test[:,0]
Y_test[:,1] =2*Y_test[:,1]
Y_test[:,2] =3*Y_test[:,2]
Y_test[:,3] =4*Y_test[:,3]
Y_test[:,4] =5*Y_test[:,4]
Y_test = np.sum(Y_test,axis=1).tolist()
mat = np.zeros((5,5))
for i in range(0,len(Y_predict)):
    mat[int(Y_predict[i])-1][Y_test[i]-1]=mat[int(Y_predict[i])-1][Y_test[i]-1]+1

In [43]:
mat

array([[1.997e+03, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 1.000e+00, 7.000e+00, 1.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 1.380e+02, 1.843e+03, 1.960e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, 1.840e+02, 1.633e+03]])

In [35]:
mat.tolist()

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1997.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 7.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 138.0, 1843.0, 196.0],
 [0.0, 0.0, 0.0, 0.0, 184.0, 1633.0]]

In [45]:
# part b
if(params[0] == 1):
    w_initial = np.ones(28*5).reshape(28, 5)
    w_initial = (w_initial.T - w_initial.T[0]).T
    costAr = []
    costAr.append(0)
    k = int(params[3])
    l = X_train.shape[0]
    for j in range(1, int(params[2])):
        for i in range(0,k):
            grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
            w_initial = w_initial+params[1]*grad 
            w_initial = (w_initial.T - w_initial.T[0]).T
        c = cost(w_initial,X_train,Y_train)
        print((j,c), end="\r", flush=True)
        costAr.append(c)
        if(costAr[j-1]-costAr[j]<.00001 and j>10):
            print(costAr[j],costAr[j-1])
            break
if(params[0] == 2):
    w_initial = np.ones(28*5).reshape(28, 5)
    w_initial = (w_initial.T - w_initial.T[0]).T
    costAr = []
    costAr.append(0)
    k = int(params[3])
    l = X_train.shape[0]
    for j in range(1, int(params[2])):
        for i in range(0,k):
            grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
            w_initial = w_initial+(params[1]/np.sqrt(j))*grad 
            w_initial = (w_initial.T - w_initial.T[0]).T
        c = cost(w_initial,X_train,Y_train)
        print((j,c), end="\r", flush=True)
        costAr.append(c)
        if(costAr[j-1]-costAr[j]<.00001 and j>10):
            print(costAr[j],costAr[j-1])
            break

0.20015944084507528 0.2001694406943409


In [ ]:
w_initial = np.random.rand(28, 5)
w_initial = (w_initial.T - w_initial.T[0]).T
# w_temp = np.zeros(28*5).reshape(28,5)
k =1
l = X_train.shape[0]
costAr = []
for j in range(1, 10000):
    for i in range(0,k):
        grad  = gradient(X_train[int((l/k)*i):int((l/k)*(i+1)),:] , Y_train[int((l/k)*i):int((l/k)*(i+1)),:], w_initial)
        grad = gradient(X_train,Y_train,w_initial)
        w_initial = w_initial+(params[1]/((l/k)))*grad  
        w_initial = (w_initial.T - w_initial.T[0]).T
    w_temp = w_initial
    l = cost(w_temp,X_train,Y_train)
    print((j,l), end="\r", flush=True)
    costAr.append(l)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(0, len(costAr[1000:2000])),costAr[1000:2000])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(0, len(costAr)),costAr)
plt.show()

In [ ]:
Y_predict[:,0] =1*Y_predict[:,0]
Y_predict[:,1] =2*Y_predict[:,1]
Y_predict[:,2] =3*Y_predict[:,2]
Y_predict[:,3] =4*Y_predict[:,3]
Y_predict[:,4] =5*Y_predict[:,4]
Y_predict = np.sum(Y_predict,axis=1).tolist()
predict = []
for i in range(0,len(Y_predict)):
    if(Y_predict[i]==1):
        predict.append("not_recom")
    elif(Y_predict[i]==2):
        predict.append("recommend")
    elif(Y_predict[i]==3):
        predict.append("very_recom")
    elif(Y_predict[i]==4):
        predict.append("priority")
    elif(Y_predict[i]==5):
        predict.append("spec_prior")

Y_predict = pd.DataFrame(predict)
Y_predict.to_csv(data_dir+sys.argv[4], header=False, index=False)

In [ ]:
w_initial = pd.DataFrame(w_initial)
w_initial.to_csv(data_dir+sys.argv[5], header=False, index=False)
# print(accuracy)

In [ ]:
Y_predict

In [ ]:
np.nonzero(w_initial)[0].shape

In [ ]:
print(features.shape,w_initial.shape)

In [ ]:
WmulX = np.exp(np.matmul(X_train,w_initial))
denom = np.sum(WmulX,axis=1)
Y_predict = np.divide(WmulX,denom.reshape(6000,1))
Y_predict[Y_predict>0.5]=1
Y_predict[Y_predict<0.5]=0
accuracy = np.trace(np.matmul(Y_predict,Y_train.T))/(6000)
print(("   accuracy="+str(accuracy*100)+"%"),end="\r",)

In [ ]:
WmulX = np.exp(np.matmul(X_test,w_initial))
denom = np.sum(WmulX,axis=1)
Y_predict = np.divide(WmulX,denom.reshape(600,1))
Y_predict[Y_predict>=0.5]=1
Y_predict[Y_predict<0.5]=0

In [ ]:
Y_predict*Y_test

In [ ]:
Y_predict.astype("int")

In [ ]:
np.trace(np.matmul(Y_predict,Y_test.T))

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# print(features.shape,labels.shape)
# pyplot.bar(range(0,len(er)),[er,erMin])
df1 = pd.DataFrame({'Y_predict': (Y_predict)[:100,1], 'Y_test': Y_test[:100,1]})
df1.plot(kind='bar',figsize=(30,30))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()